# Environment Settings

In [1]:
!pip install -r requirements.txt -q

# MLFLOW Experiment

In [2]:
import os

import mlflow
from mlflow.models import infer_signature

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

import joblib

In [3]:
# experiment setting
EXPERMENT_NAME='iris_pred'

existing_exp = mlflow.get_experiment_by_name(EXPERMENT_NAME)
if not existing_exp:
    mlflow.create_experiment(EXPERMENT_NAME, "s3://"+os.getenv('MLFLOW_BUCKET_NAME')+"/")
mlflow.set_experiment(EXPERMENT_NAME)

<Experiment: artifact_location='s3://mlflow/', creation_time=1690353973615, experiment_id='2', last_update_time=1690353973615, lifecycle_stage='active', name='iris_pred', tags={}>

In [4]:
from func.iris_classes import iris_classes

X, y = load_iris(return_X_y=True, as_frame=True)
params = {"C": 1.0, "random_state": 42}
classifier = LogisticRegression(**params).fit(X, y)

predictions = classifier.predict(X)
iris_classes(predictions)[:5]

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['setosa', 'setosa', 'setosa', 'setosa', 'setosa']

In [5]:
with open('custom_model.joblib', 'wb+') as f: 
    joblib.dump(classifier, f)

In [9]:
class CustomPredict(mlflow.pyfunc.PythonModel):
    """Custom pyfunc class used to create customized mlflow models"""
    def load_context(self, context):
        import joblib
        
        self.model = joblib.load(context.artifacts["custom_model"])

    def predict(self, context, model_input):
        from func.iris_classes import iris_classes
        
        prediction = self.model.predict(model_input)
        return iris_classes(prediction)
    

signature = infer_signature(X, predictions)
artifacts = {
    "custom_model": 'custom_model.joblib'
}
code_path = ['func/']
pip_requirements = ["-r requirements.txt"]


with mlflow.start_run():
    # log a custom model
    mlflow.pyfunc.log_model(
        artifact_path="model",
        code_path=code_path,
        artifacts=artifacts,
        python_model=CustomPredict(),
        signature=signature,
        pip_requirements=pip_requirements
    )

# Model Deployment

In [7]:
run_id = mlflow.last_active_run().info.run_id
print('last run id =', run_id)
loaded_model = mlflow.pyfunc.load_model(f"runs:/{run_id}/model")

last run id = c8841f41b59246b0a07fd7188251c45c


In [8]:
loaded_model.predict(X)[:5]

['setosa', 'setosa', 'setosa', 'setosa', 'setosa']